In [1]:
import numpy as np
import pandas as pd
import torch
# get the image from the dataset
import torch
from PIL import Image
import torchvision.transforms as transforms

In [2]:
# get the images in images folder, and save them in tensorfor folder in os.listdir('images'):
import os
import cv2
i = 0
# create folder for the tensors if it does not exist
if not os.path.exists('images/tensors'):
    os.makedirs('images/tensors')
for folder in os.listdir('images'):
    if folder == '.DS_Store':
        continue
    for file in os.listdir('images/' + folder):
        if file == '.DS_Store':
            continue
        img = cv2.imread('images/' + folder + '/' + file)
        print(f'image name: {file}')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # resize the image to 120x120
        img = cv2.resize(img, (64, 64))
        # save the image
        cv2.imwrite('images/tensors/' + str(i) + '.jpg', img)
        transform = transforms.Compose([
            transforms.ToTensor()
        ])

        img = transform(img)
        img.requires_grad = True
        print(f'img shape: {img.shape}')
        img = img.unsqueeze(0)
        if i == 0:
            images = img
        else:
            images = torch.cat((images, img), 0)
        i += 1



image name: tang-style.png
img shape: torch.Size([3, 64, 64])
image name: minimalist-gothic.png
img shape: torch.Size([3, 64, 64])
image name: traditional-house.png
img shape: torch.Size([3, 64, 64])
image name: fantasy-unfurnished.png
img shape: torch.Size([3, 64, 64])
image name: wood-elf.png
img shape: torch.Size([3, 64, 64])
image name: troll-hut.png
img shape: torch.Size([3, 64, 64])
image name: spruce-medieval.png
img shape: torch.Size([3, 64, 64])
image name: fantasy-staircase.png
img shape: torch.Size([3, 64, 64])
image name: nordic-unfurnished.png
img shape: torch.Size([3, 64, 64])
image name: priests-rest.png
img shape: torch.Size([3, 64, 64])
image name: fantasy-gatehouse.png
img shape: torch.Size([3, 64, 64])
image name: unfurnished-medieval.png
img shape: torch.Size([3, 64, 64])
image name: medieval-grocery.png
img shape: torch.Size([3, 64, 64])
image name: fantasy-window.png
img shape: torch.Size([3, 64, 64])
image name: medieval-rustic.png
img shape: torch.Size([3, 64, 6

KeyboardInterrupt: 

In [2]:
import numpy as np

def normalize_image(image):
    # Normalize the image to the range [-1, 1]

    normalize = transforms.Compose([
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
])
    normalized_image = normalize(image)
    return normalized_image

def interval_mapping(image, from_min, from_max, to_min, to_max):
    # map values from [from_min, from_max] to [to_min, to_max]
    # image: input array
    from_range = from_max - from_min
    to_range = to_max - to_min
    scaled = np.array((image - from_min) / float(from_range), dtype=float)
    return to_min + (scaled * to_range)

def denormalize_image(normalized_image):
    # Denormalize the image from the range [-1, 1] to [0, 255]
    denormalize = transforms.Compose([
        # Denormalize from [-1, 1] to [0, 255]

        transforms.Normalize(mean=[-1], std=[2]),

])
    normal = denormalize(normalized_image)
    return normal


In [3]:
def train_discriminator(real,fake,optimizer,criterion):
    # train the discriminator
    # real is the real image, fake is the fake image
    # optimizer is the optimizer for the discriminator
    # return the loss of the discriminator
    # freeze the generator
    optimizer.zero_grad()
    real_loss = criterion(discriminator(real), torch.ones(real.shape[0], 1))
    fake_loss = criterion(discriminator(fake), torch.zeros(fake.shape[0], 1))
    loss = real_loss + fake_loss
    loss.backward()
    optimizer.step()
    return loss

In [4]:
def train_generator(optimizer,fake_data,criterion):
    optimizer.zero_grad()
    loss = criterion(discriminator(fake_data), torch.ones(fake_data.shape[0], 1))
    loss.backward()
    optimizer.step()
    return loss


In [5]:
from torch import nn
class Discriminator(nn.Module):
    def __init__(self,channels=3):
        # simple discriminator, input is batch_size * 64x64x3, output is batch_size * 1
        super(Discriminator, self).__init__()
        self.channels = channels
        self.conv1 = nn.Conv2d(self.channels, 16, 4, 2, 1)
        self.leaky_relu = nn.LeakyReLU(0.2)
        self.batch_norm1 = nn.BatchNorm2d(16)

        self.conv2 = nn.Conv2d(16, 32, 4, 2, 1)
        self.batch_norm2 = nn.BatchNorm2d(32)

        self.conv3 = nn.Conv2d(32, 64, 4, 2, 1)
        self.batch_norm3 = nn.BatchNorm2d(64)

        self.conv4 = nn.Conv2d(64, 128, 4, 2, 1)
        self.batch_norm4 = nn.BatchNorm2d(128)

        self.conv5 = nn.Conv2d(128, 1, 4, 1, 0)
        self.flatten = nn.Flatten()

        self.sigmoid = nn.Sigmoid()



    def forward(self, x):
        x = self.conv1(x)
        x = self.leaky_relu(x)
        x = self.batch_norm1(x)
        x = self.conv2(x)
        x = self.leaky_relu(x)
        x = self.batch_norm2(x)
        x = self.conv3(x)
        x = self.leaky_relu(x)
        x = self.batch_norm3(x)
        x = self.conv4(x)
        x = self.leaky_relu(x)
        x = self.batch_norm4(x)
        x = self.conv5(x)
        x = self.flatten(x)
        x = self.sigmoid(x)

        return x


In [6]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, z_dim=100, channels=3, img_size=64):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        self.channels = channels
        self.img_size = img_size

        self.linear = nn.Linear(self.z_dim, 128 * (self.img_size // 16) ** 2)
        self.deconv1 = nn.ConvTranspose2d(128, 64, 4, 2, 1)
        self.batch_norm1 = nn.BatchNorm2d(64)

        self.deconv2 = nn.ConvTranspose2d(64, 32, 4, 2, 1)
        self.batch_norm2 = nn.BatchNorm2d(32)

        self.deconv3 = nn.ConvTranspose2d(32, 16, 4, 2, 1)
        self.batch_norm3 = nn.BatchNorm2d(16)

        self.deconv4 = nn.ConvTranspose2d(16, self.channels, 4, 2, 1)

        self.tanh = nn.Tanh()

    def forward(self, z):
        x = self.linear(z)
        x = x.view(x.size(0), 128, (self.img_size // 16), (self.img_size // 16))
        x = self.deconv1(x)
        x = self.batch_norm1(x)
        x = self.deconv2(x)
        x = self.batch_norm2(x)
        x = self.deconv3(x)
        x = self.batch_norm3(x)
        x = self.deconv4(x)
        x = self.tanh(x)
        return x

In [7]:
from torch.utils.tensorboard import SummaryWriter
# create folder iamges/taining if not exist
import os
if not os.path.exists('images/augmented'):
    os.makedirs('images/augmented')
writer = SummaryWriter('runs/GAN')
epochs = 200
batch_size = 64
lr = 0.0002
z_dim = 100
channels = 3
img_size = 64
"""device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')"""
device = torch.device('cpu')



generator = Generator(z_dim, channels, img_size).to(device)
discriminator = Discriminator(channels).to(device)
criterion = nn.BCELoss()
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr)

In [8]:
import os
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image

# Define data transformation (e.g., resize and normalize)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Adjust mean and std as needed
])

# Create an instance of the custom dataset
custom_dataset = CustomDataset(root_dir='images/augmented', transform=transform)

# Create a data loader
batch_size = 32
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)



In [14]:
# check if the data loader works
import matplotlib.pyplot as plt
import numpy as np
for i, data in enumerate(data_loader):
    # check max and min values of the pixels
    print(f'max: {data.max()}, min: {data.min()}')

max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0
max: 1.0, min: -1.0


In [ ]:
import torchvision.utils as vutils
from torchvision.utils import save_image
for epoch in range(epochs):
    print(f'epoch: {epoch}')
    for i, data in enumerate(data_loader):
        real = data.to(device)

        noise = torch.randn(batch_size, z_dim).to(device)
        fake = generator(noise)
        loss_D = train_discriminator(real, fake, optimizer_D, criterion)
        noise = torch.randn(batch_size, z_dim).to(device)
        fake = generator(noise)
        loss_G = train_generator(optimizer_G, fake, criterion)
        # log loss values to TensorBoard
        writer.add_scalar('loss_D', loss_D, epoch * len(data_loader) + i + 1)
        writer.add_scalar('loss_G', loss_G, epoch * len(data_loader) + i + 1)
        # log generated images to TensorBoard
        if i % 50 == 0:
            print(f"adding images to tensorboard")

            fake_to_save = denormalize_image(fake)
            print(f'max: {fake.max()}, min: {fake.min()}')
            print(f'max: {real.max()}, min: {real.min()}')
            img_grid_real = vutils.make_grid(real[:32], normalize=True)
            img_grid_fake = vutils.make_grid(fake_to_save[:32], normalize=True)
            # check max and min values of the pixels
            print(f'max: {img_grid_real.max()}, min: {img_grid_real.min()}')
            print(f'max: {img_grid_fake.max()}, min: {img_grid_fake.min()}')

            writer.add_image('real_images', img_grid_real, epoch * len(data_loader) + i + 1)
            writer.add_image('fake_images', img_grid_fake, epoch * len(data_loader) + i + 1)
        if i % 50 == 0:
            print('Epoch: %d, Batch: %d, loss_D: %.3f, loss_G: %.3f' % (epoch, i, loss_D, loss_G))
            fake = denormalize_image(fake)

            save_image(fake, f'images/training/fake_images-{epoch * len(data_loader) + i + 1}.png')

epoch: 0
adding images to tensorboard
max: 0.9991971254348755, min: -0.9995982050895691
max: 1.0, min: -1.0
max: 1.0, min: 0.0
max: 1.0, min: 0.0
Epoch: 0, Batch: 0, loss_D: 1.485, loss_G: 0.805
adding images to tensorboard
max: 0.9993756413459778, min: -0.9986878037452698
max: 1.0, min: -1.0
max: 1.0, min: 0.0
max: 1.0, min: 0.0
Epoch: 0, Batch: 50, loss_D: 0.261, loss_G: 2.606
adding images to tensorboard
max: 0.9998615384101868, min: -0.9994674324989319
max: 1.0, min: -1.0
max: 1.0, min: 0.0
max: 1.0, min: 0.0
Epoch: 0, Batch: 100, loss_D: 0.134, loss_G: 3.773
adding images to tensorboard
max: 0.9997698068618774, min: -0.9995294809341431
max: 1.0, min: -1.0
max: 1.0, min: 0.0
max: 1.0, min: 0.0
Epoch: 0, Batch: 150, loss_D: 0.208, loss_G: 4.670
adding images to tensorboard
max: 0.9999839663505554, min: -0.9999112486839294
max: 1.0, min: -1.0
max: 1.0, min: 0.0
max: 1.0, min: 0.0
Epoch: 0, Batch: 200, loss_D: 0.121, loss_G: 4.614
adding images to tensorboard
max: 0.9999397397041321, 

In [13]:
# perform data augmentation to increase the size of the dataset
from keras.preprocessing.image import ImageDataGenerator
import os

# Create an instance of the ImageDataGenerator with desired augmentations
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load your 500 images and iterate through them
# Apply augmentations and save the augmented images to a new directory
input_dir = 'images/tensors'
output_dir = 'images/augmented'
# Create output directory if it does not exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i, data in enumerate(data_loader):
    real = data.to(device)
    print(f"before normalization {real.shape}")
    print(f"after normalization {real.shape}")
    # iterate through the images in the batch and apply augmentation
    for j in range(real.shape[0]):
        # Reshape to array of 1 sample
        x = real[j, :, :, :].detach().cpu().numpy()
        #x = normalize_image(x)
        """x = x.view(64,64,3)"""
        x= x.transpose(1,2,0)
        x = x.reshape((1,) + x.shape)
        print(x.shape)
        # Generate 10 augmented images using the above datagen
        i = 0
        for batch in datagen.flow(x, batch_size=1, save_to_dir=output_dir, save_prefix='aug', save_format='png'):
            i += 1
            if i > 50:
                break  # otherwise the generator would loop indefinitely


before normalization torch.Size([32, 3, 64, 64])
after normalization torch.Size([32, 3, 64, 64])
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
before normalization torch.Size([32, 3, 64, 64])
after normalization torch.Size([32, 3, 64, 64])
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64,